# Setup

In [3]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import numpy as np
import os
import time

## Load Data

In [4]:
lines = []
# need errors='ignore' to get rid of some utf-8 errors when decoding
with open('rockyou.txt', 'r', encoding='utf-8', errors='ignore') as data_file:
    for line in data_file:
        lines.append(line.rstrip())
print(len(lines))

14344391


### Split Data

In [5]:
test_proportion = 0.2
validation_proportion = 0.1
test_set_count = (int) (test_proportion * len(lines))
validation_set_count = (int) (validation_proportion * len(lines))


X_train, X_test = train_test_split(lines, test_size=test_set_count, random_state=1)
X_train, X_val = train_test_split(X_train, test_size=validation_set_count, random_state=1)

print(f'{len(X_train)=} ({round(len(X_train) / len(lines), 2) * 100}%), {len(X_test)=} ({round(len(X_test) / len(lines), 2) * 100}%), {len(X_val)=} ({round(len(X_val) / len(lines), 2) * 100}%)')

len(X_train)=10041074 (70.0%), len(X_test)=2868878 (20.0%), len(X_val)=1434439 (10.0%)


# Markov Chain Model

Borrowed from [here](https://github.com/brannondorsey/markov-passwords/tree/master)

In [6]:
import pickle

# X_train = lines
stats = {}
max_ngrams = 3
for idx, line in enumerate(X_train):
	X_train[idx] += '\n' # to make the next part work

# create a list of ngrams from a single line in
# the training data
def get_ngram(line, n):
	output = []
	for i, char in enumerate(line):
		# use backticks as start of line characters
		# e.g. test == "```t... ``te... `tes... test" for 4grams
		if i - n < 0:
			buff = ''
			for j in range(abs(i - n)):
				buff += '`'
			buff += line[0:i]
			output.append((buff, char))
		else:
			output.append((line[i - n:i], char))
	return output

for line in X_train:
    # add ngrams to the stats dict for all n less than or
    # equal to max_ngrams (unigrams, bigrams, trigrams, etc...)
	# line = line + '\\n'
	for i in range(max_ngrams):
		for gram in get_ngram(line, i + 1):
			prev = gram[0] # previous characters, ngram
			nxt = gram[1] # next character
			# if this ngram hasn't been seen yet
			# add it to the stats dict
			if not prev in stats:
				stats[prev] = {}
			# if the next character hasn't been seen to
			# follow the ngram yet, add it the ngram's 
			# dict of seen characters
			if not nxt in stats[prev]:
				stats[prev][nxt] = 0
			# increment the statistic
			stats[prev][nxt] += 1

# convert frequency counts to probabilities
for ngram in stats:
	
	chars = []
	occur = []
	probs = []

	for key, value in stats[ngram].items():
		chars.append(key)
		occur.append(value)

	total = sum(occur)
	probs = [float(x) / float(total) for x in occur]

	for key, value in stats[ngram].items():
		stats[ngram][key] = float(value) / float(total)


In [7]:
print(X_train[0])
print(get_ngram(X_train[0], 4))
# print(stats)

melody2005

[('````', 'm'), ('```m', 'e'), ('``me', 'l'), ('`mel', 'o'), ('melo', 'd'), ('elod', 'y'), ('lody', '2'), ('ody2', '0'), ('dy20', '0'), ('y200', '5'), ('2005', '\n')]


In [8]:
import pickle
import time
import numpy as np

max_ngrams = 3 # ngram size
num_generate = 10 # number of passwords to generate

# generate a single new password using a stats dict
# created during the training phase 
def gen_password(n):
	output = '`' * n
	for i in range(100):
		output += gen_char(output[i:i + n])
		if output[-1] == '\n':
			return output[0:-1].replace('`', '')[0:-1]

# Sample a character if the ngram appears in the stats dict.
# Otherwise recursively decrement n to try smaller grams in
# hopes to find a match (e.g. "off" becomes "of").
# This is a deviation from a vanilla markov text generator
# which one n-size. This generator uses all values <= n.
# preferencing higher values of n first. 
import random
def gen_char(ngram):
	if ngram in stats:
		# sample from the probability distribution
		return random.choices(list(stats[ngram].keys()), weights=stats[ngram].values(), k=1)[0]
		# return np.random.choice(stats[ngram].keys(), p=stats[ngram].values())
	else:
		# print('{} not in stats dict'.format(ngram))
		return gen_char(ngram[0:-1])

# with open('data/{}-gram.pickle'.format(max_ngrams)) as file:
# 	stats = pickle.load(file)

# start = time.time()

for i in range(num_generate):
	pw = gen_password(max_ngrams)
	if pw is not None:
		print(pw)

# print('finished in {:.2f} seconds'.format(time.time() - start))

hubberstanson4
GsmHjW5
sanijeff080
kozmine
gie8
amthewr
xuelle
1253
eb7r2bewe
hij7774r1chien


# Password Proposal Evaluation

I will try to get the markov chain model to estimate the probability that a password appears in RockYou based on its transition weights.

In [9]:
random.seed(0)
example_password = gen_password(1)
# print(example_password)
# print(list(stats['h']))
# stats[ngram][key] = float(value) / float(total)
probability = stats['`'][example_password[0]]
for idx, first_letter in enumerate(example_password):
    next_letter_idx = idx + 1
    if next_letter_idx < len(example_password):
        probability *= stats[first_letter][example_password[next_letter_idx]]

print(probability)
# print(stats['h']['u'])
# print(list(stats.keys())[:5])

7.005967640109821e-10
